# Multiprocessing



In [186]:
import sys
import os

sys.path.insert(0, "python")

import multiprocessing as mp
import numpy as np
import matplotlib.pyplot as plt
from ishigami import *
from functools import partial

from auxiliary import * 
rslt = list()
for _ in range(100):
    x = np.random.uniform(low=-np.pi, high=np.pi, size=3)
    rslt.append(ishigami(x))

In [163]:

num_outer, num_inner = 100, 100

stat_1 = get_conditional_variance_fast(num_outer, num_inner, which)
stat_2 = get_conditional_variance_readable(num_outer, num_inner, which)      

np.testing.assert_equal(stat_1, stat_2)

In [212]:
num_outer, num_inner = 1000, 1000

%timeit get_conditional_variance_fast(num_outer, num_inner, which)
%timeit get_conditional_variance_readable(num_outer, num_inner, which)

9.19 s ± 692 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.78 s ± 355 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [190]:
# Uncertainty propagation
num_draws = 10

%timeit get_unconditional_variance_fast(num_draws)
%timeit get_unconditional_variance_readable(num_draws)

171 µs ± 4.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
105 µs ± 2.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


We first want to get a sense how many CPU's we have available.

In [4]:
print(f"Number of cpu : {mp.cpu_count()}")

Number of cpu : 4


In [238]:
inputs =  np.random.uniform(low=-np.pi, high=np.pi, size=(num_draws, 3))

def ishigami_readable(x, a=0.7, b=0.1):
    return np.sin(x[0]) + a * np.sin(x[1]) ** 2 + b * x[2] ** 4 * np.sin(x[0])

def ishigami_fast(x, a=0.7, b=0.1):
    x0, x1, x2 = x[:, 0], x[:, 1], x[:, 2]
    return np.sin(x0) + a * np.sin(x1) ** 2 + b * x2 ** 4 * np.sin(x0)

rslt = list()
for input_ in inputs:
    rslt.append(ishigami_readable(input_))

np.testing.assert_equal(ishigami_fast(inputs), rslt)

## mp.Process

In [210]:
num_outer, num_inner = 1000, 1000

import multiprocessing

def task(num_outer, num_inner, which):
    
    print(f"... started on input parameter {which}")
    rslt = get_conditional_variance_fast(num_outer, num_inner, which)
    print(f"... finished input parameter {which}")

    return rslt

if __name__ == '__main__':
    jobs = []
    for which in range(3):
        p = mp.Process(target=task, args=(num_outer, num_inner, which))
        jobs.append(p)
        p.start()

... started on input parameter 0
... started on input parameter 1
... started on input parameter 2
... finished input parameter 1
... finished input parameter 2
... finished input parameter 0


## mp.Pool

In [213]:
pool = mp.Pool(processes = 3)
task_partial = partial(task, num_outer, num_inner)
pool.map(task_partial, range(3))

pool.close()
pool.join()

... started on input parameter 1
... started on input parameter 0
... started on input parameter 2
... finished input parameter 2
... finished input parameter 0
... finished input parameter 1


## References

* https://www.journaldev.com/15631/python-multiprocessing-example#python-multiprocessing-process-queue-and-locks

* https://pymotw.com/2/multiprocessing/basics.html